### Stage 2

Run three experiments and return scores for both datasets.

In [ ]:
import keras
import numpy as np
import util
import model
import config
from model_rnn import RNNAutoencoder

In [ ]:
def train_eval_ensemble_model(rnn_train_data, rnn_test_datas, xgb_train_data, xgb_test_datas, xgb_cv_folds, config):
    """
    Trains an ensemble model on the provided data, returns trained RNN and XGB models, CV score and test scores.
    Used for stage 2, experiments 1 and 2.
    
    Steps:
    (1) Trains a RNN autoencoder on @rnn_train_data, then uses generated embeddings as additional input to
    the XGB (basically, appends the predictions to the @xgb_cv_data).
    (2) Trains an ensemble XGB model using the provided folds @xgb_cv_folds. Reports the CV score.
    (3) Fr testing, first generates RNN embeddings for each of @rnn_test_data. Appends them to @xgb_test_data
    and reports the test scores of the ensemble model on each of test dataset.

    Args:
        rnn_train_data:   np.array; 
                          sequential data for training the RNN autoencoder (without labels)
        rnn_test_datas:   list of np.array;
                          list of sequential data for generating test predictions with RNN autoencoder
        xgb_train_data:   tuple (np.array, np.array);
                          data for training and validation of the XGBoost (ensemble) model
        xgb_test_datas:   list of tuple (np.array, np.array);
                          array of datasets used for testing
        xgb_cv_folds:     list of tuple (np.array, np.array);
                          cross-validation folds used for training and validation of the XGBoost model
        config:           object;
                          configuration object

    Returns:
        rnn_model:        RNNAutoencoder;
                          trained recurrent model
        ensemble_model:   xgboost.XGBClassifier;
                          trained ensemble (XGBoost) model
        cv_score:         list(3);
                          array length 3 with the mean and CI for the CV AUC score on the provided folds
        test_scores:      list of list(3);
                          array of length which is equal to len(@xgb_test_data), with the means and CIs (based on
                          100 runs with different random seeds) of AUC scores for every test dataset provided
    """
    
    # Train RNN
    rnn_model = LSTM_Autoencoder(config.RNN_OPTIMIZER, config.RNN_LOSS, config.RNN_LSTM_SIZE, config.RNN_DENSE_SIZE)
    rnn_model.fit(rnn_train_data, epochs=config.RNN_EPOCHS_MAIN, batch_size=config.RNN_BATCH_SIZE)
    
    # Append RNN embeddings to XGBoost train data
    rnn_train_embeddings = rnn_model.encode(rnn_train_data).numpy()
    xgb_train_data = (
        np.concatenate([xgb_train_data[0], rnn_train_embeddings.reshape(-1, 1)], axis=1),
        xgb_train_data[1]
    )
    
    # Append RNN embeddings to XGBoost test data
    for i in range(len(xgb_test_datas)):
        test_embedding = rnn_model.encode(rnn_test_datas[i]).numpy()
        xgb_test_datas[i] = (
            np.concatenate([xgb_test_datas[i][0], test_embedding.reshape(-1, 1)], axis=1),
            xgb_test_datas[i][1]
        )
    
    # Calculate ensemble XGBoost CV score
    cv_score = model.calc_xgb_cv_auc(config.XGB_PARAMS, xgb_train_data[0], xgb_train_data[1], xgb_cv_folds)
    
    # Calculate ensemble XGBoost test scores
    test_scores = []
    for xgb_test_data in xgb_test_datas:
        scores, ensemble_model = model.calc_xgb_auc_based_on_multiple_runs_and_return_model(
            config.XGB_PARAMS, xgb_train_data[0], xgb_train_data[1], xgb_test_data[0], xgb_test_data[1])
        test_scores.append(scores)
    
    return rnn_model, ensemble_model, cv_score, test_scores

In [ ]:
def finetune_eval_ensemble_model(rnn_model, xgb_model_loc, rnn_tune_data, rnn_test_data, xgb_tune_data,
                                 xgb_test_data, xgb_cv_folds, config):
    """
    Fine-tunes given ensemble model on the provided data, returns CV score and test scores.
    Used for stage 2, experiment 3.
    
    Steps:
    (1) Fine-tunes a RNN autoencoder on @rnn_tune_data, then uses generated embeddings as additional input to
    the XGB (basically, appends the predictions to the @xgb_tune_data).
    (2) Fine-tunes an ensemble XGB model using the provided folds @xgb_cv_folds. Reports the CV score.
    (3) For testing, first generates RNN embeddings for each of @rnn_test_data. Appends them to @xgb_test_data
    and reports the test scores of the ensemble model on each of test dataset.

    Args:
        rnn_model:        RNNAutoencoder; 
                          pre-trained RNN model
        xgb_model_loc:    str; 
                          location of the saved pre-trained XGBoost model
        rnn_tune_data:    np.array; 
                          sequential data for fine-tuning the RNN autoencoder (without labels) (COVID)
        rnn_test_data:    np.array;
                          sequential data for generating test predictions with RNN autoencoder (non-COVID test)
        xgb_tune_data:    tuple (np.array, np.array);
                          data for fine-tuning of the XGBoost (ensemble) model                 (COVID)
        xgb_test_data:    list of tuple (np.array, np.array);
                          data for testing of the XGBoost (ensemble) model                     (non-COVID test)
        xgb_tune_folds:   list of tuple (np.array, np.array);
                          folds used for fine-tuning and evaluation of the ensemble model      (COVID folds)
        config:           object;
                          configuration object

    Returns:
        cv_score:         list(3);
                          array length 3 with the mean and CI for the CV AUC score of the fine-tuned model
                          on COVID dataset using the provided @xgb_test_folds folds
        test_score:       list(3);
                          array of length which is equal to len(@xgb_test_data), with the means and CIs of AUC
                          scores of the fine-tuned model on the non-COVID test set
    """
    
    # Fine-tune RNN
    rnn_model.fit(rnn_tune_data, epochs=config.RNN_EPOCHS_MAIN, batch_size=config.RNN_BATCH_SIZE)
    
    # Append generated RNN embeddings to XGB tune data
    rnn_tune_embeddings = rnn_model.encode(rnn_tune_data).numpy()
    xgb_tune_data = (
        np.concatenate([xgb_tune_data[0], rnn_tune_embeddings.reshape(-1, 1)], axis=1),
        xgb_tune_data[1]
    )
    
    # Append generated RNN embeddings to XGB test data
    rnn_test_embeddings = rnn_model.encode(rnn_test_data).numpy()
    xgb_test_data = (
        np.concatenate([xgb_test_data[0], rnn_test_embeddings.reshape(-1, 1)], axis=1),
        xgb_test_data[1]
    )

    # Fine-tune and evaluate XGBoost (ensemble) model
    scores_covid = []
    for fold in xgb_tune_folds:
        covid_train = (xgb_tune_data[0][fold[0]], xgb_tune_data[1][fold[0]])
        covid_test = (xgb_tune_data[0][fold[1]], xgb_tune_data[1][fold[1]])
        scores_covid.append(model.fine_tune_and_calc_auc_based_on_multiple_runs(xgb_model_loc, config.XGB_PARAMS,
                                                      covid_train[0], covid_train[1],
                                                      covid_test[0], covid_test[1]))
    score_covid = model.calc_mean_and_confidence_interval(scores_covid)
        
    # Fine-tune on all COVID and evaluate on non-COVID
    score_noncovid = model.fine_tune_and_calc_auc_based_on_multiple_runs(xgb_model_loc, config.XGB_PARAMS,
                                                                        xgb_tune_data[0] xgb_tune_data[1],
                                                                        xgb_test_data[0], xgb_test_data[1])
    
    return score_covid, score_noncovid

In [ ]:
# Load data for RNN
noncovid_rnn_train_data = util.load_data_rnn(config.NONCOVID_RNN_TRAIN_DATA_LOC)
noncovid_rnn_test_data = util.load_data_rnn(config.NONCOVID_RNN_TEST_DATA_LOC)
covid_rnn_data = util.load_data_rnn(config.COVID_RNN_DATA_LOC)

# Load data for XGBoost
_, x, y, _ = util.load_data(config.NONCOVID_XGB_TRAIN_DATA_LOC)
noncovid_xgb_train_data = (x, y)
_, x, y, _ = util.load_data(config.NONCOVID_XGB_TEST_DATA_LOC)
noncovid_xgb_test_data = (x, y)
_, x, y, _ = util.load_data(config.COVID_XGB_DATA_LOC)
covid_xgb_data = (x, y)

# Load CV folds
noncovid_xgb_cv_folds = util.load_folds(noncovid_xgb_train_data, config.NONCOVID_CV_SPLITS_LOC)
covid_xgb_cv_folds = util.load_folds(covid_xgb_train_data, config.COVID_CV_SPLITS_LOC)

In [ ]:
# Experiment 1

rnn_model, ensemble_model, score_covid_experiment1, score_noncovid_experiment1 = train_eval_ensemble_model(
    noncovid_rnn_train_data,
    [noncovid_rnn_test_data, covid_rnn_data],
    noncovid_xgb_train_data,
    [noncovid_xgb_test_data, covid_xgb_data],
    noncovid_xgb_cv_folds,
    config
)
ensemble_model.save_model('ensemble_model_dump')

In [ ]:
# Experiment 2

_, _, score_covid_experiment2, score_noncovid_experiment2 = train_eval_ensemble_model(
    covid_rnn_data,
    [noncovid_rnn_test_data],
    covid_xgb_data,
    [noncovid_xgb_test_data],
    covid_xgb_cv_folds,
    config
)

In [ ]:
# Experiment 3

_, _, score_covid_experiment3, score_noncovid_experiment3 = finetune_eval_ensemble_model(
    rnn_model,
    'ensemble_model_dump',
    covid_rnn_data,
    noncovid_rnn_test_data,
    covid_xgb_data,
    noncovid_xgb_test_data,
    covid_xgb_cv_folds,
    config
)

In [ ]:
print(score_covid_experiment1, score_noncovid_experiment1)

In [ ]:
print(score_covid_experiment2, score_noncovid_experiment2)

In [ ]:
print(score_covid_experiment3, score_noncovid_experiment3)